In [1]:
# Load necessary modules
import cobra
import FM as FM
from cobrapy_bigg_client import client
import multiprocessing as mp
cobra.Configuration().solver='gurobi'

In [2]:
# Load model
identifier="iJO1366"
model=cobra.io.web.load.load_model(identifier)

Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "ibsen"


In [3]:
# Set the identifiers of importan reactions
biomass="BIOMASS_Ec_iJO1366_core_53p95M"
o2="EX_o2_e"
ethanol="EX_etoh_e"

In [ ]:
# Make a copy of the model
modelo=model.copy()

# Force the model to work in anaerobic mode
modelo.reactions.get_by_id(o2).bounds=[0,0]

#Identify the maximum growth
maximum=modelo.slim_optimize()

# Force the model to have a minimum growth = 0.5 * maximum

modelo.reactions.get_by_id(biomass).bounds=[0.1,1000]

# Identify genes that are essential for this minimum growth and
# remove them from the model 
gMCSsGrowth=FM.createGMCS(modelo,1)
protected=set(g for case in gMCSsGrowth[1] for g in case)
cobra.manipulation.delete.remove_genes(modelo,protected,False)

# Block the production of ethanol

modelo.reactions.get_by_id(ethanol).bounds=[0,0]

# Set the maximum length of the gMCSs and compute them
maxLength=4
gMCSs=FM.createGMCS(modelo,maxLength)

In [5]:
# Auxiliar functions to check if a gMCS induces couple growth

def analyze(start,end):    
    CSs=set()
    for caso in myList[start:end]:
        with modelo:
            for g in caso:
                modelo.genes.get_by_id(g).knock_out()
            sol=modelo.slim_optimize(error_value=-1)
            if not sol==-1:
                CSs.add(caso)
    return(CSs)

def analyze2(start,end):    
    CSs=set()
    for caso in myList[start:end]:
        with modelo:
            
            for g in caso:
                modelo.genes.get_by_id(g).knock_out()
            sol=modelo.slim_optimize(error_value=-1)
            if sol>0:
                CSs.add(caso)
    return(CSs)

In [8]:
# Check which GMCSs induce couple growth by checking
# 1.- That they do not prevent growth
# 2.- After stablishing the addecuate lower bound for growth
#     there is a minimum ethanol production

modelo=model.copy()
modelo.reactions.get_by_id(o2).bounds=[0,0]
modelo.reactions.get_by_id(biomass).bounds=[0.1,1000]

myList=set()

for C in gMCSs:
    myList.update(C)
myList=list(myList)

cpus=64
k=0
argus=[]
num=1000

while k*num<=len(myList):
    argus.append((k*num,(k+1)*num+1))
    k+=1

with mp.Pool(processes=cpus) as mp_pool:
    results=mp_pool.starmap(analyze,argus)

notEssentials=set()
for result in results:
    notEssentials.update(result)
    
myList=list(notEssentials)
modelo.objective=modelo.reactions.get_by_id(ethanol)
modelo.objective_direction="min"
cpus=64
k=0
argus=[]
num=1000

while k*num<=len(myList):
    argus.append((k*num,(k+1)*num+1))
    k+=1

with mp.Pool(processes=cpus) as mp_pool:
    results=mp_pool.starmap(analyze2,argus)

Interventions=set()
for result in results:
    Interventions.update(result)
    

Read LP format model from file /tmp/tmpjqf0_z55.lp
Reading time = 0.01 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
615


In [9]:
print("Total of interventions:",len(Interventions))
for i in range(1,maxLength+1):
    print("Interventions of length",i,len(set(C for C in Interventions if len(C)==i)))

Total of interventions: 67
Interventions of length 1 0
Interventions of length 2 3
Interventions of length 3 64
